# Graph Partitioning with a QUBO objective function

In [128]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from graphs import DrawSolution
from graphs import FileToNetwork
from graphs import DrawNetwork
from graphs import GraphPartitioning

We will try to explore the GP problem using the QUBO model, in which the objective function is given by

$$ E(\mathbf{x}) = \sum_i Q_{ii} x_i + \sum_{i < j} Q_{ij} x_i x_j $$

where $x_i \in \{0, +1 \}$. We will try to minimize  this function or, equivalently, to find

$$ \min_{\mathbf{x}}{\mathbf{x}^T Q \mathbf{x}} $$

where $\mathbf{x} \in \{0, +1 \}^n$.

Given a graph $G = (V, E)$ with vertex set $V$ and edge set $E$ such that $n = |V|$ and $m = |E|$, it can be shown that in the GP problem the matrix $Q$ is as follows


$$Q_{ij} = \begin{cases}
        \alpha - \beta , \quad &(i, j) \in E\\
        \alpha, \quad &(i, j) \notin E, \, i \neq j\\
        \beta g_i - \alpha(n-1), \quad &i = j
    \end{cases}$$

where $g_i$ is de degree of the vertex $i$ and 

$$\frac{\alpha}{\beta} \geq \frac{\min(2 \Delta, n)}{8}$$

where $\Delta$ is the maximal degree of the graph. We will study this problem for different values of $\alpha$ and $\beta$.

In [2]:
def QMatrix(fileName, alpha, beta):
    '''
    This function returns the Q matrix for a given graph
    
    Input
    -----
    fileName  : string
                name of the file containing the graph
    alpha     : double
                weight for the balancing constraint
    beta      : double
                weight for the minimum edge cut size
    
    Output
    ------
    Q  : array
         Q matrix for the QUBO optimization problem
    '''
    
    vertex = []
    gi = []
    
    #Getting the information of the graph
    with open(fileName) as file:
        
        next(file) #discard line
        n, m = [int(x) for x in next(file).split()]
        
        for i in range(n):
            vertex.append([])
            gi.append([])
        
        #Degrees
        i = 0   
        for line in file:
            e = line.split()
            gi[i] = len(e)
            for j in range(len(e)):
                vertex[i].append(int(e[j]))
            i += 1
            

        #Building the Q matrix
        Q = np.full((n,n), alpha)
        
        for i in range(n):
            Q[i][i] = beta*gi[i] - alpha*(n-1)
        for i in range(len(vertex)):
            for j in range(len(vertex[i])):
                Q[i][vertex[i][j]-1] = alpha - beta
    return Q 

In [198]:
def QUBOSolution(Q):
    '''
    Finds the solutions of the minimization problem
    
    Input
    -----
    Q  : array
         Q matrix for the QUBO optimization problem
    
    Output
    ------
    num_sols  : double
                number of solutions         
    xmin      : array
                solutions to the GP problem
    '''
    
        
    #Preparing all possible vectors
    x = itertools.product([0,1], repeat = len(Q))
    list_x = list(x)
    
    #Verifying the number of combinations
    if (len(list_x)) != 2**len(Q):
        print("Not all vectors found \n")
        return None
    
    #Finding the solution
    sol = np.zeros(len(list_x))
    aux = 0
    for i in range(len(list_x)):
        aux = np.matmul(list_x[i], Q)
        sol[i] = np.matmul(aux, list_x[i])
        
    
    E = np.amin(sol)    
    index = np.where(sol == E)
    num_sols = len(index[0])
    
    xmin = []
    for i in range(len(index[0])):
        j = index[0][i]
        xmin.append(list_x[j])
                
    return num_sols, xmin

In [197]:
def CheckIfEqual(fileName, xmin):
    '''
    Checks if any solution is equal to the solution found by METIS
    
    Input
    -----
    fileName  : string
                name of the file with the METIS solution
    xmin      : array
                QUBO solutions
    '''
    
    metis = []
    #Reading METIS solution
    with open(fileName) as file:
        for line in file:
            i = line.split()
            metis.append(int(i[0]))
    metis = np.array(metis)
    
    #Comparing
    for i in range(len(xmin)):
        sol = np.array(xmin[i])
        
        if (metis == sol).all():
            return True
        
    #Not found
    return False

## First example

We will try to first to solve the problem using the following graph
<img src="example1.png" width="500" height="340">

Now $\frac{\alpha}{\beta} \geq 0.75$. However, we will fix $\beta = 1$ and we will choose $\alpha \in [0, 1.5]$ to study how this parameters afect the result.

In [192]:
beta = 1
alpha_values = np.linspace(0, 1.5, 15)

In [191]:
#METIS solution
GraphPartitioning("example1.txt", 2)

******************************************************************************
METIS 5.0 Copyright 1998-13, Regents of the University of Minnesota
 (HEAD: , Built on: Nov  5 2021, 13:09:47)
 size of idx_t: 32bits, real_t: 64bits, idx_t *: 64bits

Graph Information -----------------------------------------------------------
 Name: example1.txt, #Vertices: 6, #Edges: 5, #Parts: 2

Options ---------------------------------------------------------------------
 ptype=kway, objtype=cut, ctype=shem, rtype=greedy, iptype=metisrb
 dbglvl=0, ufactor=1.030, no2hop=NO, minconn=NO, contig=NO, nooutput=NO
 seed=-1, niter=10, ncuts=1

Direct k-way Partitioning ---------------------------------------------------
 - Edgecut: 1, communication volume: 2.

 - Balance:
     constraint #0:  1.000 out of 0.333

 - Most overweight partition:
     pid: 0, actual: 3, desired: 3, ratio: 1.00.

 - Subdomain connectivity: max: 1, min: 1, avg: 1.00

 - Each partition is contiguous.

Timing Information -------------

In [226]:
print("alpha" + "     " + "num solutions" + "      " + "equal" + "           " + "solutions",)
for alpha in alpha_values:
    Q =QMatrix("example1.txt", alpha, beta)
    num_sols, xmin = QUBOSolution(Q)
    solution = CheckIfEqual("example1.txt.part.2", xmin)
    print("{0:.2f}".format(alpha), num_sols, solution, xmin, sep = "            ")

alpha     num solutions      equal           solutions
0.00            2            False            [(0, 0, 0, 0, 0, 0), (1, 1, 1, 1, 1, 1)]
0.11            1            False            [(1, 1, 1, 1, 1, 1)]
0.21            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.32            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.43            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.54            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.64            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.75            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.86            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
0.96            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
1.07            2            True            [(0, 0, 0, 1, 1, 1), (1, 1, 1, 0, 0, 0)]
1.18            2

## Second example